In [1]:
from tg.common.ml import batched_training as bt
from tg.grammar_ru import *
from tqdm import tqdm
import pandas as pd
import numpy as np
import torch

FOLDER = Loc.data_cache_path/'Rowling'

In [2]:
class TuneWithTrainedNetwork(torch.nn.Module):
    def __init__(self, Wtrained: torch.Tensor, Btrained: torch.Tensor):
        super(TuneWithTrainedNetwork, self).__init__()
        self.Wtrained = torch.nn.Embedding.from_pretrained(Wtrained)
        self.Btrained = torch.nn.Embedding.from_pretrained(Btrained)
        self.W = torch.nn.Embedding(num_embeddings=Wtrained.shape[0], embedding_dim=Wtrained.shape[1])
        self.b = torch.nn.Embedding(num_embeddings=Btrained.shape[0], embedding_dim=Btrained.shape[1])
        
    def forward(self, batch):
        Wtr = self.Wtrained(torch.LongTensor(batch['input'].pretrained.values))
        btr = self.Btrained(torch.LongTensor(batch['input'].pretrained.values))
        W = self.W(torch.LongTensor(batch['input'].trainable.values))
        b = self.b(torch.LongTensor(batch['input'].trainable.values))
        return torch.sum((Wtr * W), dim=1).unsqueeze(1) + btr + b
    
class NonceOnlyNetwork(torch.nn.Module):
    def __init__(self, W: torch.nn.Embedding, W_c: torch.nn.Embedding, b: torch.nn.Embedding, b_c: torch.nn.Embedding):
        super(NonceOnlyNetwork, self).__init__()
        self.W = W
        self.W_c = W_c
        self.b = b
        self.b_c = b_c
        
    def forward(self, batch):
        W_c = self.W_c(torch.LongTensor(batch['input'].context.values))
        b_c = self.b_c(torch.LongTensor(batch['input'].context.values))
        W = self.W(torch.LongTensor(batch['input'].word.values))
        b = self.b(torch.LongTensor(batch['input'].word.values))
        return torch.sum((W_c * W), dim=1).unsqueeze(1) + b_c + b
    
def train_loop(model, loss_fn, optimizer, idb, batcher):
    t_loss = 0
    n = batcher.get_batch_count(idb)
    for batch_index in tqdm(range(n)):
        batch = batcher.get_batch(idb, batch_index)
        pred = model(batch)
        loss = loss_fn(pred, torch.tensor(batch['label'].values, dtype=torch.float32))
        
        optimizer.zero_grad()
        loss.backward()
        t_loss += loss.item()
        optimizer.step()
    return t_loss / n

def train(name_idb, network, input_names, train_name):
        total_loss = []
        optimizer = torch.optim.Adam(network.parameters(), lr=0.001)
        input_extractor = bt.PlainExtractor.build('input').index().apply(take_columns=input_names)
        label_extractor = bt.PlainExtractor.build('label').index().apply(take_columns='label')
        bchr = bt.Batcher(batch_size=4000, extractors=[input_extractor, label_extractor])
        epochs = 300
        for t in range(epochs):
            print(f"Epoch {t+1}\n-------------------------------")
            loss = train_loop(network, torch.nn.MSELoss(), optimizer, name_idb, bchr)
            total_loss.append(loss)
        print(train_name, "done!")
        return total_loss
    
def get_idb_network(df, w, bias):
    name_db = DataBundle(index_frame=df)
    name_idb = bt.IndexedDataBundle(
        index_frame = df,
        bundle = name_db
    )
    network = TuneWithTrainedNetwork(torch.Tensor(w.values), torch.Tensor(bias.values).unsqueeze(1))
    return name_idb, network
    
def tune_nfs(tune_nfs, vectors_df):
    w_ = vectors_df.iloc[:, :300]
    bias_ = vectors_df.iloc[:, 300]
    w_c = vectors_df.iloc[:, 301:601].reset_index(drop=True)
    bias_c = vectors_df.iloc[:, 601]

    nonce_in_regular_context = (common_cooc[common_cooc['nf_index'].isin(tune_nfs) & 
                                            ~common_cooc['context_nf_index'].isin(tune_nfs)]
                                .rename(columns={'nf_index': 'trainable', 'context_nf_index': 'pretrained'}))
    idb, network = get_idb_network(nonce_in_regular_context, w_c, bias_c)
    loss_1 = train(idb, network, ['pretrained', 'trainable'], 'Nonce in regular context')
    W_emb = network.W
    b_emb = network.b
    
    regular_in_nonce_context = (common_cooc[~common_cooc['nf_index'].isin(tune_nfs) & 
                                            common_cooc['context_nf_index'].isin(tune_nfs)]
                                .rename(columns={'nf_index': 'pretrained', 'context_nf_index': 'trainable'}))
    idb, network = get_idb_network(regular_in_nonce_context, w_, bias_)
    loss_2 = train(idb, network, ['pretrained', 'trainable'], 'Regular in nonce context')
    W_c_emb = network.W
    b_c_emb = network.b
    
    nonce_in_nonce_context = (common_cooc[common_cooc['nf_index'].isin(tune_nfs) & 
                                          common_cooc['context_nf_index'].isin(tune_nfs)]
                              .rename(columns={'nf_index': 'word', 'context_nf_index': 'context'}))
    nonce_db = DataBundle(index_frame=nonce_in_nonce_context)
    idb = bt.IndexedDataBundle(
        index_frame = nonce_in_nonce_context,
        bundle = nonce_db
    )
    network = NonceOnlyNetwork(W_emb, W_c_emb, b_emb, b_c_emb)
    loss_3 = train(idb, network, ['word', 'context'], 'Nonce in nonce context')
    
    name_vecs = np.concatenate([network.W(torch.LongTensor(tune_nfs)).detach().numpy(),
                                network.b(torch.LongTensor(tune_nfs)).detach().numpy(),
                                network.W_c(torch.LongTensor(tune_nfs)).detach().numpy(),
                                network.b_c(torch.LongTensor(tune_nfs)).detach().numpy()], axis=1)
    nvdf = np.array(vectors_df.copy())
    nvdf[tune_nfs] = name_vecs
    return (pd.DataFrame(nvdf), 
            [nonce_in_regular_context, regular_in_nonce_context, nonce_in_nonce_context], 
            [loss_1, loss_2, loss_3])

In [3]:
vectors_file =FOLDER/'glove'/'output.txt'
vdf = pd.read_csv(vectors_file, sep=' ',header=None).set_index(0)
vdf = vdf.reset_index(drop=True).iloc[:-1]
vdf.columns=list(range(vdf.shape[1]))
vdf.index.name='index'

tuning_candidates = pd.read_csv(Loc.data_cache_path/'tuning_candidates.csv', sep='\t').set_index('nf_n')
cooc_pairs = pd.read_csv(FOLDER/'Rowling_cooc.cvs').drop(columns='Unnamed: 0')
common_cooc = cooc_pairs[['nf_index', 'context_nf_index', 'label']]

In [4]:
all_freq_nnf = tuning_candidates[tuning_candidates.in_freq_prop | 
                                 tuning_candidates.is_name_by_cnt | 
                                 tuning_candidates.is_nonce_by_freq].index.values
tuned_df, inputs, losses = tune_nfs(all_freq_nnf, vdf)

Epoch 1
-------------------------------


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 71/71 [00:21<00:00,  3.34it/s]


Epoch 2
-------------------------------


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 71/71 [00:18<00:00,  3.83it/s]


Epoch 3
-------------------------------


...

Epoch 143
-------------------------------


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 71/71 [00:13<00:00,  5.15it/s]


Epoch 144
-------------------------------


 58%|██████████████████████████████████████████████████████████████████████████████████████████▋                                                                  | 41/71 [00:09<00:07,  4.18it/s]


RuntimeError: [enforce fail at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10\core\impl\alloc_cpu.cpp:81] data. DefaultCPUAllocator: not enough memory: you tried to allocate 24916800 bytes.

In [ ]:
tuned_df